In [69]:
import requests
import json
import pandas as pd
import sqlite3
import os

# Diretório onde o arquivo será salvo
directory = "../datasets"
csv_directory = os.path.join(directory, "csv")  # Novo diretório para os CSVs

# Verifica se o diretório CSV já existe, caso contrário, cria o diretório
if not os.path.exists(csv_directory):
    os.makedirs(csv_directory)

class etBcb:
    def __init__(self, api_link):
        self.api_link = api_link
        self.dados = None
        self.df = None

    def requisicao_api(self):
        """
        Método GET para a API e armazenar a resposta.
        """
        try:
            resposta = requests.get(self.api_link)
            if resposta.status_code == 200:
                self.dados = resposta.json()
                print('Status Code:', resposta.status_code)
            else:
                print('Erro na requisição. Status Code:', resposta.status_code)
        except Exception as e:
            print('Erro ao fazer a requisição:', e)

    def detectar_estrutura_json(self):
        """
        Método para detectar a estrutura do JSON.
        """
        # Verifica se a chave 'resultados' e 'series' existem
        if isinstance(self.dados, list) and 'resultados' in self.dados[0] and 'series' in self.dados[0]['resultados'][0]:
            return 'nova_serie'
        else:
            # Depuração: imprimir a estrutura do JSON para verificar o que está errado
            print("Estrutura JSON não reconhecida. Aqui está o JSON retornado:")
            print(json.dumps(self.dados, indent=4))  # Imprime o JSON formatado para leitura
            return 'desconhecida'

    def transformar_dados(self):
        """
        Método para transformar os dados JSON em um DataFrame pandas.
        """
        if self.dados:
            try:
                estrutura = self.detectar_estrutura_json()

                if estrutura == 'nova_serie':
                    # Processamento para a nova estrutura com 'series'
                    data_series = self.dados[0]['resultados'][0]['series'][0]['serie']
                    # Transformando a série de dados em um DataFrame
                    self.df = pd.DataFrame(list(data_series.items()), columns=['Data', 'Valor'])
                    print('Transformação concluída para a nova estrutura com series.')
                else:
                    print('Estrutura JSON desconhecida.')
                    
            except KeyError as e:
                print(f'Chave não encontrada no JSON: {e}')
            except Exception as e:
                print('Erro ao transformar os dados:', e)
        else:
            print('Nenhum dado para transformar.')

    def salvar_sqlite(self, nome_tabela):
        """
        Método para salvar o DataFrame transformado em um banco de dados SQLite.
        """
        nome_banco = os.path.join(directory, 'Fecomdb.db')
        if self.df is not None:
            try:
                with sqlite3.connect(nome_banco) as conexao:
                    self.df.to_sql(nome_tabela, conexao, if_exists='replace', index=False)
                    print(f'Dados salvos na tabela "{nome_tabela}" do banco de dados "{nome_banco}".')
            except Exception as e:
                print('Erro ao salvar os dados no banco de dados SQLite:', e)
        else:
            print('Nenhum dado para salvar no banco de dados.')

    def salvar_csv(self, nome_arquivo):
        """
        Método para salvar o DataFrame transformado em um arquivo CSV.
        """
        if self.df is not None:
            try:
                self.df.to_csv(os.path.join(csv_directory, nome_arquivo), sep=';', decimal=',', encoding='utf-8-sig', index=False)
                print(f'Dados salvos no arquivo CSV "{nome_arquivo}" na pasta "csv".')
            except Exception as e:
                print('Erro ao salvar o CSV:', e)
        else:
            print('Nenhum dado no CSV.')


In [77]:
url = "https://servicodados.ibge.gov.br/api/v3/agregados/8695/periodos/201101-202407/variaveis/11623?localidades=N1[all]&classificacao=11046[56726]|12355[56724]"
    
    # Nome da tabela
name_table = 'PMSCargaBrasil'

    # Instanciar a classe ETL com o link da API
etl = etBcb(url)

    # Executar o método de requisição para extrair os dados da API
etl.requisicao_api()

    # Transformar os dados
etl.transformar_dados()

    # Salvar os dados no SQLite e no CSV
etl.salvar_sqlite(name_table)
etl.salvar_csv(f'{name_table}.csv')

Status Code: 200
Transformação concluída para a nova estrutura com series.
Dados salvos na tabela "PMSCargaBrasil" do banco de dados "../datasets\Fecomdb.db".
Dados salvos no arquivo CSV "PMSCargaBrasil.csv" na pasta "csv".


In [36]:
import pandas as pd

url ='https://servicodados.ibge.gov.br/api/v3/agregados/8688/periodos/201102-202407/variaveis/11623?localidades=N1[all]&classificacao=11046[56726]|12355[107071,106869,106874,31399,106876,31426]'
data = pd.read_json(url)
print(data['resultados'][0][0]['classificacoes'][0])

{'id': '11046', 'nome': 'Tipos de índice', 'categoria': {'56726': 'Índice de volume de serviços'}}
